<a href="https://colab.research.google.com/github/vsnupoudel/Flask/blob/final/LfTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import random

In [4]:
!mkdir '/content/data/'

mkdir: cannot create directory ‘/content/data/’: File exists


In [5]:
train= pd.read_csv('/content/data/training.csv', delimiter=';', decimal=',')
test= pd.read_csv('/content/data/validation.csv', delimiter=';', decimal=',')
# random.seed(12)
train.sample(3)

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,classLabel
2957,a,17.83,0.001100,u,-0.010785,0.692820,1.000,t,t,11,f,g,0.0,3000,0.0,NaN,1,yes.
2102,a,22.83,0.000229,u,-1.238763,0.615668,2.290,t,t,7,t,g,140.0,2384,1400000.0,t,1,yes.
3602,a,41.00,0.000204,y,1.122596,0.799766,0.125,t,t,23,t,g,455.0,1236,4550000.0,NaN,1,yes.


In [6]:
test.sample(3)

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,classLabel
115,b,42.00,0.000979,u,0.748151,1.181493,7.96,t,t,8,f,g,0.0,0,0.0,t,0,yes.
178,b,43.00,0.000029,y,1.201602,0.789425,1.75,t,t,8,f,g,100.0,375,1000000.0,NaN,1,yes.
173,NaN,20.42,0.000750,u,0.525192,0.609564,1.50,t,t,1,f,g,160.0,234,1600000.0,t,0,yes.


In [8]:
train.to_csv('/content/data/train.csv')
test.to_csv('/content/data/test.csv')


# Extending pandas clas
[Reference for extending pandas api](https://pandas.pydata.org/pandas-docs/stable/development/extending.html)

In [9]:
@pd.api.extensions.register_dataframe_accessor("hlst")
class HighLevelStats():
    def __init__(self, pObject):
        self._pObject = pObject
    
    @property    
    def print_stats_numeric(self):
        return self._pObject.describe(include= np.number).transpose()
    
    @property    
    def print_stats_object(self):
        return self._pObject.describe(include= np.object).transpose()

In [10]:
train.hlst.print_stats_numeric

,count,mean,std,min,25%,50%,75%,max
v2,3661.0,3.282071e+01,1.266618e+01,13.750000,23.000000,2.867000e+01,4.083000e+01,8.025000e+01
v3,3700.0,5.847872e-04,5.402007e-04,0.000000,0.000150,4.250000e-04,9.625000e-04,2.800000e-03
v5,3700.0,-5.331808e-02,2.871240e+00,-4.996939,-2.516349,-5.278621e-02,2.406251e+00,4.999274e+00
v6,3700.0,7.841879e-01,1.365138e-01,0.273945,0.693130,7.878424e-01,8.760522e-01,1.215066e+00
v7,3700.0,3.439496e+00,4.335229e+00,0.000000,0.500000,1.750000e+00,5.000000e+00,2.850000e+01
v10,3700.0,4.160000e+00,6.750553e+00,0.000000,0.000000,2.000000e+00,6.000000e+00,6.700000e+01
v13,3600.0,1.626950e+02,1.560457e+02,0.000000,0.000000,1.200000e+02,2.800000e+02,1.160000e+03
v14,3700.0,2.246706e+03,8.708571e+03,0.000000,0.000000,1.130000e+02,1.059750e+03,1.000000e+05
v15,3600.0,1.626950e+06,1.560457e+06,0.000000,0.000000,1.200000e+06,2.800000e+06,1.160000e+07
v17,3700.0,9.254054e-01,2.627716e-01,0.000000,1.000000,1.000000e+00,1.000000e+00,1.000000e+00


In [11]:
train.hlst.print_stats_object

,count,unique,top,freq
v1,3661,2,b,2382
v4,3636,3,u,3055
v8,3700,2,t,3171
v9,3700,2,t,2239
v11,3700,2,f,1924
v12,3700,3,g,3430
v16,1555,2,t,1023
classLabel,3700,2,yes.,3424


In [12]:
class HLS(pd.DataFrame): 
    @property    
    def print_stats_numeric(self):
        return self.describe(include= np.number).transpose()
    
    @property    
    def print_stats_object(self):
        return self.describe(include= np.object).transpose()

In [13]:
tt = HLS(train)

In [14]:
tt.print_stats_numeric

,count,mean,std,min,25%,50%,75%,max
v2,3661.0,3.282071e+01,1.266618e+01,13.750000,23.000000,2.867000e+01,4.083000e+01,8.025000e+01
v3,3700.0,5.847872e-04,5.402007e-04,0.000000,0.000150,4.250000e-04,9.625000e-04,2.800000e-03
v5,3700.0,-5.331808e-02,2.871240e+00,-4.996939,-2.516349,-5.278621e-02,2.406251e+00,4.999274e+00
v6,3700.0,7.841879e-01,1.365138e-01,0.273945,0.693130,7.878424e-01,8.760522e-01,1.215066e+00
v7,3700.0,3.439496e+00,4.335229e+00,0.000000,0.500000,1.750000e+00,5.000000e+00,2.850000e+01
v10,3700.0,4.160000e+00,6.750553e+00,0.000000,0.000000,2.000000e+00,6.000000e+00,6.700000e+01
v13,3600.0,1.626950e+02,1.560457e+02,0.000000,0.000000,1.200000e+02,2.800000e+02,1.160000e+03
v14,3700.0,2.246706e+03,8.708571e+03,0.000000,0.000000,1.130000e+02,1.059750e+03,1.000000e+05
v15,3600.0,1.626950e+06,1.560457e+06,0.000000,0.000000,1.200000e+06,2.800000e+06,1.160000e+07
v17,3700.0,9.254054e-01,2.627716e-01,0.000000,1.000000,1.000000e+00,1.000000e+00,1.000000e+00


In [15]:
tt.print_stats_object

,count,unique,top,freq
v1,3661,2,b,2382
v4,3636,3,u,3055
v8,3700,2,t,3171
v9,3700,2,t,2239
v11,3700,2,f,1924
v12,3700,3,g,3430
v16,1555,2,t,1023
classLabel,3700,2,yes.,3424


# Tensorflow StatGen maybe better

In [17]:
!mkdir '/content/pipeline/'
!mkdir '/content/data/'

mkdir: cannot create directory ‘/content/data/’: File exists


In [19]:
_data_root

'/content/data/'

## Install TfDV, might take upto 5 minutes

In [34]:
import tensorflow_data_validation as tfdv

In [40]:
train_stats = tfdv.generate_statistics_from_csv(data_location='/content/data/train.csv')
tfdv.visualize_statistics(train_stats)